In [ ]:
# import requests 

# base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
# relative_url = '03-vector-search/eval/documents-with-ids.json'
# docs_url = f'{base_url}/{relative_url}?raw=1'
# docs_response = requests.get(docs_url)
# documents = docs_response.json()

In [1]:
import json

In [6]:
with open ("../03-vector-search/eval/documents-with-ids.json", "rt") as f_out:
    documents = json.load(f_out)

In [9]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

## Load ground truth

In [11]:
import pandas as pd

# base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
# relative_url = '03-vector-search/eval/ground-truth-data.csv'
# ground_truth_url = f'{base_url}/{relative_url}?raw=1'

# df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = pd.read_csv("../03-vector-search/eval/ground-truth-data.csv") 
print(df_ground_truth.shape)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
print(df_ground_truth.shape)
ground_truth = df_ground_truth.to_dict(orient='records')

(4631, 3)
(1842, 3)
